In [62]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [2]:
# form mzp6.ipnyb
def draw_system(method, t_0, y_0s, approx, exact = None, labels=None, step = 0.1):
    count = np.ceil(10.0/step)
    X = np.linspace(t_0,t_0+10,count)
    Y = np.zeros(shape=(len(approx),X.shape[0]))
    if exact:
        RY = np.asarray([y(X) if y is not None else None for y in exact])
    Y[:,0] = np.asarray(y_0s)
    
    for i in np.arange(1,count):
        Y_updates = method(tn=X[i-1],yn=Y[:,i-1],functions=approx,h=step)
        if isinstance(Y_updates,tuple):
            Y[:,i] = Y[:,i-1] + Y_updates[1]
        else:
            Y[:,i] = Y[:,i-1] + Y_updates
    
    figure_count = len(approx)
    if exact:
        figure_count += len(filter(lambda x: x is not None,exact))
    figure = 1
    plt.figure(figsize = (9,figure_count*3))
    for i in range(len(approx)):
        ax = plt.subplot(figure_count,1,figure)
        if labels and labels[i]:
            ax.set_title(labels[i])
        plt.plot(X,Y[i], color = 'blue', label ='Approx')
        if exact and exact[i]:
            plt.plot(X,RY[i], color = 'red', linestyle='--', label='Exact')
        plt.legend(loc = 'upper left')
        figure += 1

        if exact and exact[i]:
            plt.subplot(figure_count,1,figure)
            plt.plot(X,np.abs(RY[i]-Y[i]), color = 'blue', label= 'Global error')
            plt.legend(loc = 'upper left')
            figure+=1
            
    plt.show()

In [3]:
# form mzp6.ipnyb
class SystemSolver():
    def __init__(self,updates = None):
        if isinstance(updates,list):
            max_len = max([len(x) for x in updates])
            [x.extend([0]*(max_len-len(x))) for x in updates]
            self.updates = np.asarray(updates, dtype = np.float64)
        else:
            self.updates = updates
        
    @property
    def update_count(self):
        return min(self.updates.shape)-1
    
    def __call__(self,tn,yn,functions,h):
        k = np.zeros(shape=(self.update_count,len(functions)))
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]
            upd = np.sum(b[:,np.newaxis]*k,axis = 0)
            for j in range(len(functions)):
                k[i,j]= h * functions[j](tn+a*h, yn+upd)
        c = self.updates[self.update_count,1:]
        if self.updates.shape[0]>self.updates.shape[1]:
            c_= self.updates[self.update_count+1,1:]
            return np.sum(c[:,np.newaxis]*k, axis = 0), np.sum(c_[:,np.newaxis]*k, axis = 0)
        return np.sum(c[:,np.newaxis]*k, axis = 0)

In [4]:
fehlberg_4_5 = SystemSolver([[0],
                            [1/4,1/4],
                            [3/8,3/32,9/32],
                            [12/13,1932/2197,-7200/2197,7296/2197],
                            [1,439/216,-8,3680/513,-845/4104],
                            [1/2,-8/27,2,-3544/2565,1859/4104,-11/40],
                            [0,25/216,0,1408/2565,2197/4104,-1/5,0],
                            [0,16/135,0,6656/12825,28561/56430,-9/50,2/55]])
fehlberg_4_5.ranks = (4,5)

In [5]:
def get_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    return rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]

Dwie spreżyny

In [6]:
k_1 = 4
k_2 = 10
L = 3
m = 3

In [7]:
x_0 = 0
v_0 = 3

In [8]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1 - k_2)/m*ys[0]
]

In [9]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_0,v_0],
            approx = functions,
            labels = ["x(t)","v(t)"])

In [10]:
Y = np.zeros(shape=(2,))

In [11]:
def double_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        spring = get_spring([-10,0],[new_x,0],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,0],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        mass.set_data([new_x],[0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [12]:
k_1 = 5
k_2 = 10
L = 3
m = 3

In [13]:
x_0 = 2
v_0 = 3

In [14]:
double_spring(method=fehlberg_4_5,
               y_0s=[x_0,v_0],
               approx= functions,
               step=0.05)

In [15]:
def triple_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    spring3, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    points = (-15,-5,5,15)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        spring3.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, spring3, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        fixed_1 = points[0]
        mass_1 = points[1]+Y[0]
        mass_2 = points[2]+Y[2]
        fixed_2 = points[3]
        
        spring = get_spring([fixed_1,0],[mass_1,0],6)
        spring1.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_1,0],[mass_2,0],6)
        spring2.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_2,0],[fixed_2,0],6)
        spring3.set_data(spring[0],spring[1])
        
        mass.set_data([mass_1,mass_2],[0,0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, spring3, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [16]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2
]

In [17]:
k_1 = 2
k_2 = 2
k_3 = 2
m_1 = .2
m_2 = .2

In [18]:
x_1_0 = 3
v_1_0 = 0
x_2_0 = 0
v_2_0 = 0

In [19]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
            approx = functions,
            labels = ["x_1(t)","v_1(t)","x_2(t)","v_2(t)"])

In [20]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

Z oporami ($\psi(v)$) zależnymi od prędkości

$\psi(v) = c  v$

In [21]:
c = 1

In [22]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*ys[1],
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*ys[3]
]

In [23]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

$\psi(v) = sgn(v) c  v^2$

In [24]:
c = 0.1

In [25]:
x_1_0 = 20
v_1_0 = 0
x_2_0 = -10
v_2_0 = 0

In [26]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*np.sign(ys[1])*ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*np.sign(ys[3])*ys[3]**2
]

In [27]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

Dwie spręzyny - ruch w dwóch wymiarach

In [28]:
def double_spring_2d(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    
    old_x = []
    old_y = []
    
    history, = ax.plot([],[],linestyle='--')
    
    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        history.set_data([],[])
    
        return history, spring1, spring2, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        new_y = Y[2]
        old_x.append(new_x)
        old_y.append(new_y)
        spring = get_spring([-10,0],[new_x,new_y],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        mass.set_data([new_x],[new_y])
        history.set_data([old_x],[old_y])

        time_text.set_text(time_template % (t*step))
        return history, spring1, spring2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [29]:
k = 2
L = 10
m = 3
g = 9.81

In [30]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) ) )/m,
    lambda t,ys: ys[3],
    lambda t,ys: -g + 1/m * ( k* ys[2] * ( ( L - a(ys) ) / a(ys) + (L - b(ys)) / b(ys) ))
]

In [31]:
x_0 = 0
v_x_0 = 0
y_0 = 0
v_y_0 = 0

In [32]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_0,v_x_0,y_0,v_y_0],
            approx = functions,
            labels = ["x(t)","v_x(t)","y(t)","v_y(t)"])

In [33]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [34]:
x_0 = 0
v_x_0 = 15
y_0 = 0
v_y_0 = 15

In [35]:
k = 5

In [36]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

Z oporami ruchu

In [37]:
c = .05

In [38]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) ) )/m - np.sign(ys[1]) * c * ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: -g + 1/m * ( k* ys[2] * ( ( L - a(ys) ) / a(ys) + (L - b(ys)) / b(ys) )) - np.sign(ys[3]) * c * ys[3]**2
]

In [39]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

System z czterema spręzynami

In [40]:
def spring_node(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    
    old_x = []
    old_y = []
    
    history, = ax.plot([],[],linestyle='--')
    
    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    spring3, = ax.plot([],[])
    spring4, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        history.set_data([],[])
        spring3.set_data([],[])
        spring4.set_data([],[])
    
        return history, spring1, spring2, spring3, spring4, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        new_y = Y[2]
        old_x.append(new_x)
        old_y.append(new_y)
        spring = get_spring([-10,0],[new_x,new_y],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        spring = get_spring([0,10],[new_x,new_y],6)
        spring3.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[0,-10],6)
        spring4.set_data(spring[0],spring[1])
        mass.set_data([new_x],[new_y])
        history.set_data([old_x],[old_y])

        time_text.set_text(time_template % (t*step))
        return history, spring1, spring2, spring3,spring4, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [41]:
c_ = 0.2

In [42]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
c = lambda ys: np.sqrt((ys[0]) + (L + ys[2])**2)
d = lambda ys: np.sqrt((ys[0]) + (L - ys[2])**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) + \
                   ys[0] *( (L-c(ys))/c(ys) + (L-d(ys))/d(ys) ) ) )/m,
    lambda t,ys: ys[3],
    lambda t,ys: ( k*  ( (L+ys[2]) *( L - c(ys)) / c(ys) - (L-ys[2])*(L - d(ys)) / d(ys) + \
                   ys[2] *( (L-a(ys))/a(ys) + (L-b(ys))/b(ys) ) ) )/m - g
]

In [134]:
k = 25

In [44]:
x_0 = 3
v_x_0 = 0
y_0 = 3
v_y_0 = 0

In [45]:
spring_node(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [46]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
c = lambda ys: np.sqrt((ys[0]) + (L + ys[2])**2)
d = lambda ys: np.sqrt((ys[0]) + (L - ys[2])**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) + \
                   ys[0] *( (L-c(ys))/c(ys) + (L-d(ys))/d(ys) ) ) )/m - \
                   np.sign(ys[1]) * c_ * ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: ( k*  ( (L+ys[2]) *( L - c(ys)) / c(ys) - (L-ys[2])*(L - d(ys)) / d(ys) + \
                   ys[2] *( (L-a(ys))/a(ys) + (L-b(ys))/b(ys) ) ) )/m - g - \
                   np.sign(ys[3]) * c_ * ys[3]**2
]

In [47]:
spring_node(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [122]:
balls = np.mgrid[0:5,0:5]
pos = balls*10.
vel = np.zeros(shape=(2,3,3))
print vel
pos[0,2,2] = 15.
print pos
step = 0.01
plt.figure()
plt.scatter(mass[1].ravel(),mass[0].ravel())
plt.show()

mass = pos[:,1:4,1:4]
north = pos[:,0:3,1:4]
south = pos[:,2:5,1:4]
west = pos[:,1:4,0:3]
east = pos[:,1:4,2:5]

spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
spring_n = spring_len(north)
spring_s = spring_len(south)
spring_e = spring_len(east)
spring_w = spring_len(west)
update_north = (spring_n-L)*(mass-north)/spring_n
update_south = (spring_s-L)*(mass-south)/spring_s
update_east = (spring_e-L)*(mass-east)/spring_e
update_west = (spring_e-L)*(mass-east)/spring_w
update_vel = step*k*(update_north + update_south + update_west + update_east)/m
vel += update_vel
update_pos = step*vel
pos[:,1:4,1:4] += update_pos

plt.figure()
plt.scatter(mass[1].ravel(),mass[0].ravel())
plt.show()
print mass

[[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
[[[  0.   0.   0.   0.   0.]
  [ 10.  10.  10.  10.  10.]
  [ 20.  20.  15.  20.  20.]
  [ 30.  30.  30.  30.  30.]
  [ 40.  40.  40.  40.  40.]]

 [[  0.  10.  20.  30.  40.]
  [  0.  10.  20.  30.  40.]
  [  0.  10.  20.  30.  40.]
  [  0.  10.  20.  30.  40.]
  [  0.  10.  20.  30.  40.]]]
[[[ 10.          10.00416667  10.        ]
  [ 20.00093169  14.99078689  20.        ]
  [ 30.          30.00416667  30.        ]]

 [[ 10.          20.          30.        ]
  [  9.99813661  19.99824045  30.        ]
  [ 10.          20.          30.        ]]]


In [146]:
def spring_field(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global vel,pos
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    balls = np.mgrid[0:5,0:5]
    pos = balls*10.
    vel = np.zeros(shape=(2,3,3))
    pos[0,2,2] = 15.
    
    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global vel, pos
        mass = pos[:,1:4,1:4]
        north = pos[:,0:3,1:4]
        south = pos[:,2:5,1:4]
        west = pos[:,1:4,0:3]
        east = pos[:,1:4,2:5]

        spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
        spring_n = spring_len(north)
        spring_s = spring_len(south)
        spring_e = spring_len(east)
        spring_w = spring_len(west)
        update_north = (spring_n-L)*(mass-north)/spring_n
        update_south = (spring_s-L)*(mass-south)/spring_s
        update_east = (spring_e-L)*(mass-east)/spring_e
        update_west = (spring_w-L)*(mass-west)/spring_w
        update_vel = -step*k*(update_north + update_south + update_east + update_west)/m
        vel += update_vel
        update_pos = step*vel
        pos[:,1:4,1:4] += update_pos
        
        points.set_data(pos[0].ravel(),pos[1].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=20, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [147]:
spring_field()